In [7]:
years = range(1999, 2014)

headers = ['oscars', 'nominations', 'theaters', 'worldgross', 'title', 'year']
moviedata = []

In [8]:
import urllib2
import requests
from requests.exceptions import HTTPError
import re
from bs4 import BeautifulSoup
import pickle
from datetime import datetime


import numpy as np
randn = np.random.randn
from pandas import *
import pandas as pd


def text2int(textnum, numwords={}):
    if not numwords:
        numwords={}
        units = [
            "zero", "one", "two", "three", "four", "five", "six", "seven", "eight",
            "nine", "ten", "eleven", "twelve", "thirteen", "fourteen", "fifteen",
            "sixteen", "seventeen", "eighteen", "nineteen",
        ]
        for idx, word in enumerate(units):    numwords[word] = idx
    if textnum not in numwords:
          raise Exception("Illegal word: " + textnum)
    return numwords[textnum]

In [9]:
#years = [2013]

for y in years:
    url = "http://www.boxofficemojo.com/yearly/chart/?yr=" + str(y) + "&p=.htm"
    page = urllib2.urlopen(url)
    soup = BeautifulSoup(page)
    pages = (len(soup.find_all('a', href=re.compile('chart/\?page=.*&.*')))/2)+1

    for p in range(1,pages+1):
        url = "http://www.boxofficemojo.com/yearly/chart/?page=" + str(p) + \
        "&view=releasedate&view2=domestic&yr=" + str(y) + "&p=.htm"
    
        page = urllib2.urlopen(url)
        soup = BeautifulSoup(page)
    
        movielinks =[]
    
        if soup.find_all('a', href=re.compile('movies/\?id=.*.htm')):
            for link in soup.find_all('a', href=re.compile('movies/\?id=.*.htm')):
                movielink = "http://www.boxofficemojo.com" + str(link['href'])
                movielinks.append(movielink)

        movielinks = movielinks[1:]

        for m in movielinks:
            try:
                url = m
                page = urllib2.urlopen(url)
            except urllib2.HTTPError, error:
                continue
            soup = BeautifulSoup(page)
            if (soup.find(text = re.compile('Academy Awards')) and
                soup.find("title").text and
                soup.find(text=re.compile('Worldwide:')) and
                soup.find(text=re.compile('Widest.*Release')) and
                soup.find(text=re.compile('Nominated for .*'))):
                titlestr = soup.find("title").text
                title = str(titlestr.split("(")[0].strip())
                grosscell = soup.find(text=re.compile('Worldwide:')).findNext().b
                grossstr = str(grosscell)[4:-4]
                gross = int(grossstr.replace(',', ''))
                theatcell = soup.find(text=re.compile('Widest.*Release')).findNext()
                theatstr = str(theatcell)[6:-14]
                theatnum = int(theatstr.replace(',', ''))
                oscarcell = soup.find(text=re.compile('Nominated for .*'))
                oscarnomt = str(oscarcell.split(" ")[2])
                nominations = text2int(oscarnomt.lower())
                oscars = 0
                if len(oscarcell) > 30:
                    oscarawat = str(oscarcell.split(" ")[5])
                    oscars = text2int(oscarawat.lower())
        
                year = datetime(year = y, month = 12, day = 31)
                moviedict = dict(zip(headers, [oscars, nominations, theatnum, gross, title, year]))
                moviedata.append(moviedict)


In [10]:
moviesdf = DataFrame(moviedata)
print moviesdf

     nominations  oscars  theaters                                      title  \
0              3       0      3126  Star Wars: Episode I - The Phantom Menace   
1              6       0      2821                            The Sixth Sense   
2              1       0      3257                                Toy Story 2   
3              1       0      3314      Austin Powers: The Spy Who Shagged Me   
4              4       4      2903                                 The Matrix   
5              1       1      3131                                     Tarzan   
6              1       0      3411                                  The Mummy   
7              1       0      3151                              Stuart Little   
8              4       0      2875                             The Green Mile   
9              8       5      1990                            American Beauty   
10             3       1      3069                              Sleepy Hollow   
11             5       0    

In [11]:
with open('movies2.pkl', 'w') as f:
    pickle.dump(moviesdf, f)

In [12]:
with open('movies2.pkl', 'r') as f:
    moviesdf = pickle.load(f)

In [13]:
moviesdf

,nominations,oscars,theaters,title,worldgross,year
0,3,0,3126,Star Wars: Episode I - The Phantom Menace,1027044677,1999-12-31
1,6,0,2821,The Sixth Sense,672806292,1999-12-31
2,1,0,3257,Toy Story 2,485015179,1999-12-31
3,1,0,3314,Austin Powers: The Spy Who Shagged Me,312016858,1999-12-31
4,4,4,2903,The Matrix,463517383,1999-12-31
5,1,1,3131,Tarzan,448191819,1999-12-31
6,1,0,3411,The Mummy,415933406,1999-12-31
7,1,0,3151,Stuart Little,300135367,1999-12-31
8,4,0,2875,The Green Mile,286801374,1999-12-31
9,8,5,1990,American Beauty,356296601,1999-12-31


In [14]:
moviesdf.to_csv('/Users/marcolunardi/documents/doc/metis/project2/movies2.csv')